In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import onnx

class MNISTNet(nn.Module):
    def __init__(self):
        super(MNISTNet, self).__init__()
        self.conv_block1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1), # 1, 28, 28 -> 32, 28, 28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), # 32, 28, 28 -> 32, 14, 14
            nn.BatchNorm2d(32) # Reduction de l'ecart type et de la moyenne
        )
        self.conv_block2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1), #32, 14, 14 -> 64, 14, 14
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), #64, 14, 14 -> 64, 7, 7
            nn.BatchNorm2d(64)
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=64*7*7, out_features=128),  # fixed input size
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        logits = self.classifier(x)
        return logits # // probabibilities

In [ ]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

model = MNISTNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion= nn.CrossEntropyLoss().to(device)

Using device: cpu


In [ ]:
# Training loop
print("Starting training...")
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

for epoch in range(1, 6):
    model.train()
    epoch_train_loss = 0.0
    epoch_train_correct = 0
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        epoch_train_loss += loss.item() * data.size(0)
        pred = output.argmax(dim=1, keepdim=True)
        epoch_train_correct += pred.eq(target.view_as(pred)).sum().item()
        
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.6f}')
    
    epoch_train_loss /= len(train_dataset)
    epoch_train_accuracy = 100. * epoch_train_correct / len(train_dataset)

    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_dataset)
    test_accuracy = 100. * correct / len(test_dataset)

    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)

    print(
        f"Epoch {epoch}: "
        f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.2f}% | "
        f"Test Loss: {test_loss:.4f}, Test Acc: {test_accuracy:.2f}%"
    )

torch.save(model.state_dict(), 'mnist_model.pth')
print("Model saved as mnist_model.pth")

In [12]:
# Export to ONNX
model.eval()
dummy_input = torch.randn(1, 1, 28, 28)

torch.onnx.export(
    model,
    dummy_input,
    "mnist_model.onnx",
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={
        'input': {0: 'batch_size'},
        'output': {0: 'batch_size'}
    }
)

print("Model exported to ONNX format as mnist_model.onnx")

onnx_model = onnx.load("mnist_model.onnx")
onnx.checker.check_model(onnx_model)
print("ONNX model verification passed")
print("Training complete!")

Model exported to ONNX format as mnist_model.onnx
ONNX model verification passed
Training complete!
